In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

In [ ]:
all_data = final.copy()
all_data.replace("-", np.nan, inplace=True)
clean_df = all_data.dropna(subset=['Transaction Date', 'Staff No.','Staff Name', 'Gender','Site',])
grouped_df = clean_df.groupby(["Transaction Date", "Staff No.", "Staff Name", "Gender", "Site", "Ranking"]).sum().reset_index()
melted_detail_df = pd.melt(grouped_df,
                           id_vars=["Transaction Date", "Staff No.", "Staff Name", "Gender", "Site", "Ranking"],
                           value_name="Out", var_name="Item Name")
melted_detail_df = melted_detail_df[melted_detail_df['Out'] != 0]

cleaned_df = melted_detail_df.groupby(["Transaction Date", "Staff No.", "Staff Name", "Gender", "Site", "Ranking", "Item Name"])["Out"].sum().reset_index()

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

a_float = cleaned_df['Out'].apply(is_float)

cleaned_df = cleaned_df[a_float]
cleaned_df['Out'] =cleaned_df['Out'].astype(float)

split_columns = cleaned_df['Item Name'].str.split('-', expand=True)
split_columns.columns = ["Code", "Category", "Item","size", "test", "test2","test3"]
split_columns["Category"] = split_columns["Category"].str.replace('\n', ' ')
split_columns['Item_name'] = split_columns['Item'].astype(str) + " " + split_columns['size'].astype(str)
split_columns = split_columns.drop(["Item",	"size",	"test","test2",	"test3"], axis=1)

final_df = pd.concat(([split_columns, (cleaned_df.drop(['Item Name', 'Staff Name'],axis =1))]), axis=1)
extra_date = final_df[final_df['Transaction Date']=='0/1/1900'].index
final_df.drop(extra_date, inplace =True)
final_df.rename(columns ={'Staff No.':'Staff_Number', 'Transaction Date': 'Transaction_Date'},inplace =True)

In [ ]:
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Uniform") \
    .getOrCreate()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")
spark_df = spark.createDataFrame(final_df)
spark_df.printSchema()
spark_df

In [ ]:
spark_df.write.mode("overwrite").saveAsTable("milky.consolidated_uniform")